## Word2Vec 을 이용한 평점 및 메타데이터 기반 영화 추천기법

먼저 여러가지 영화 데이터를 로드합니다. <br>
영화 데이터는 MovieLens의 데이터를 사용하였습니다.

In [113]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
import random

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

rating = pd.read_csv('ratings.csv')
tag = pd.read_csv('tags.csv')
movie = pd.read_csv('movies.csv')
# 각종 데이터를 로드합니다.

rating_df = pd.DataFrame(rating)
tag_df = pd.DataFrame(tag)
movie_df = pd.DataFrame(movie)
# 데이터 프레임을 만듭니다.

tag_df.head(10)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
5,2,89774,Tom Hardy,1445715205
6,2,106782,drugs,1445715054
7,2,106782,Leonardo DiCaprio,1445715051
8,2,106782,Martin Scorsese,1445715056
9,7,48516,way too long,1169687325


In [132]:
my_array = []
tmp_array = []
tmp_for_movie_id = 60756
tmp_for_user_id = 2
from operator import itemgetter

for userId, movieId, tag, time in tag_df.values:

    if tmp_for_user_id == userId:
        if tmp_for_movie_id == movieId:
            lower_tag = tag.lower()
            data_dict = {'time': time, 'tag': lower_tag}
            tmp_array.append(data_dict)

        else:
            sorted_list = sorted(tmp_array, key=itemgetter('time'))
            sorted_tag_list = []
            for i in sorted_list:
                sorted_tag_list.append(i.get('tag'))
            my_array.append(list(sorted_tag_list))
            tmp_for_movie_id = movieId
            tmp_array.clear()
            lower_tag = tag.lower()
            tmp_array.append({'time': time, 'tag': lower_tag})

    else:
        tmp_for_user_id = userId

for i in range(0, 10):
    print(my_array[i])


['will ferrell', 'funny', 'highly quotable']
['mma', 'tom hardy', 'boxing story']
['leonardo dicaprio', 'drugs', 'martin scorsese']
['gangster', 'mafia']
['mafia', 'al pacino']
['holocaust', 'true story']
['twist ending']
['twist ending', 'anthony hopkins', 'courtroom drama']
['britpop', 'indie record label', 'music']
['dumpster diving', 'sustainability']


In [133]:
words = []
s_array = []

for separate_array in my_array:
    arr = list(separate_array)
    s_array.append(arr)
    for word in separate_array:
        words.append(word)

word2int = {}
int2word = {}

for i, word in enumerate(words):
    word2int[word] = i
    int2word[i] = word

sentences = []
for sentence in s_array:
    sentences.append(sentence)

WINDOW_SIZE = 3

data = []
for sentence in sentences:
    for idx, word in enumerate(sentence):
        for neighbor in sentence[max(idx - WINDOW_SIZE, 0): min(idx + WINDOW_SIZE, len(sentence)) + 1]:
            if neighbor != word:
                data.append([word, neighbor])

df = pd.DataFrame(data, columns=['input', 'label'])



In [139]:
ONE_HOT_DIM = len(words)


# 큰숫자 (예를들어 35, 43 등)를 원핫 인코딩 시키는 함수.
# 35 -> (0,0,0,0,.....,1,0,0,0)
# 36 -> (0,0,0,0,.....,0,1,0,0)
def to_one_hot_encoding(data_point_index):
    one_hot_encoding = np.zeros(ONE_HOT_DIM)
    one_hot_encoding[data_point_index] = 1
    return one_hot_encoding


X = []  # 입력 배열입니다.
Y = []  # 타겟단어입니다.

for x, y in zip(df['input'], df['label']):
    X.append(to_one_hot_encoding(word2int[x]))
    Y.append(to_one_hot_encoding(word2int[y]))

# 넘파이 어레이로 변경
X_train = np.asarray(X)
Y_train = np.asarray(Y)

# 학습과정을 위한 placeholder 생성
x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))

EMBEDDING_DIM = 256

# 이 두개의 값은 각각 히든레이어의 변수가 됩니다.
W1 = tf.get_variable(str(random.random()), shape=[ONE_HOT_DIM, EMBEDDING_DIM],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([1]))
L1 = tf.add(tf.matmul(x, W1), b1)

# 출력값
W2 = tf.get_variable(str(random.random()), shape=[EMBEDDING_DIM, ONE_HOT_DIM],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([1]))
prediction = tf.nn.softmax(tf.add(tf.matmul(L1, W2), b2))

# 코스트합수 : 크로스 엔트로피
loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), axis=[1]))

# 학습과정
train_op = tf.train.AdamOptimizer(0.01).minimize(loss)
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

iteration = 180

for i in range(iteration):
    sess.run(train_op, feed_dict={x: X_train, y_label: Y_train})
    if i % 10 == 0:
        print('학습 ' + str(i) + ' 현재 코스트 : ', sess.run(loss, feed_dict={x: X_train, y_label: Y_train}))

vectors = sess.run(W1 + b1)


학습 0 현재 코스트 :  7.653345


학습 10 현재 코스트 :  6.271553


학습 20 현재 코스트 :  5.253589


학습 30 현재 코스트 :  3.4170141


학습 40 현재 코스트 :  2.7840238


...

학습 130 현재 코스트 :  2.4044049


학습 140 현재 코스트 :  2.4037032


학습 150 현재 코스트 :  2.4031534


학습 160 현재 코스트 :  2.4027061


학습 170 현재 코스트 :  2.4023368


In [142]:
def euclidean_dist(vec1, vec2):
    return np.sqrt(np.sum((vec1 - vec2) ** 2))


def find_close(word_index):
    min_dist = 100
    query_vector = vectors[word_index]
    temp = []
    for index, vector in enumerate(vectors):
        if euclidean_dist(vector, query_vector) < min_dist and not np.array_equal(vector, query_vector):
            min_dist = euclidean_dist(vector, query_vector)
            min_index = index
            temp.append({'dist': min_dist, 'tag': min_index})

    temp.append({'dist': 0, 'tag': word_index})
    sorted_list = sorted(temp, key=itemgetter('dist'))
    return sorted_list


def get_tag(word_index):
    tag_list = []
    for i in find_close(word_index):
        tag_list.append(i.get('tag'))

    return tag_list


def get_distance(word_index):
    dist_list = []
    for i in find_close(word_index):
        dist_list.append(i.get('dist'))

    return dist_list


num = 4


def find_closet_tag(word):
    idx = 0
    indexes = get_tag(word2int[word])
    distances = get_distance(word2int[word])
    for i in zip(indexes, distances):
        idx = idx+1
        if idx < num:
            print('태그 : ', int2word[i[0]], ' 거리 : ', i[1])


In [143]:
test = 'thor'
print(test, ' 태그와 가장 비슷한 태그의 목록입니다.')
print('===============================')

find_closet_tag(test)

print('\n')

test = 'world war i'
print(test, ' 태그와 가장 비슷한 태그의 목록입니다.')
print('===============================')

find_closet_tag(test)

print('\n')

test = 'marvel'
print(test, ' 태그와 가장 비슷한 태그의 목록입니다.')
print('===============================')

find_closet_tag(test)

thor  태그와 가장 비슷한 태그의 목록입니다.
태그 :  thor  거리 :  0
태그 :  guardians of the galaxy  거리 :  4.803244
태그 :  dr. strange  거리 :  4.9842157


world war i  태그와 가장 비슷한 태그의 목록입니다.
태그 :  world war i  거리 :  0
태그 :  made me cry  거리 :  0.4491413
태그 :  thought-provoking  거리 :  0.4506213


marvel  태그와 가장 비슷한 태그의 목록입니다.
태그 :  marvel  거리 :  0
태그 :  mcu  거리 :  6.0355077
태그 :  adventure  거리 :  6.432714
